# Computing quantitative features for all data.
1) Linking to drive

2) Initialization

3) Computing results

## Import and linking to drive

In [1]:
## Run once to link to google drive using GCP
#!sudo apt-get install google-drive-ocamlfuse
#!sudo add-apt-repository ppa:alessandro-strada/ppa
#!sudo apt-get update
#!sudo apt-get install google-drive-ocamlfuse

In [2]:
#!google-drive-ocamlfuse -headless -id=553698332080-4ilebkr6tajj9d4on76a6sfjilo8dbob.apps.googleusercontent.com -secret=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

# Imports

In [3]:
!pip install lpips

In [4]:
import numpy as np
import os
import re
from tqdm import tqdm

import lpips
from skimage.metrics import peak_signal_noise_ratio, structural_similarity as ssim
import torch
from PIL import Image


# Linking up drive and initialization

#Initializing google drive to a folder
if not os.path.exists("./acv"):
  os.makedirs("./acv")

!google-drive-ocamlfuse acv

In [5]:
os.chdir('G:\ACVProjects\Group-revised\Codebase\AID')

In [6]:
'''
from google.colab import drive

drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/Courses/COMS 4995-6 - Applied Computer Vision/Group Project/Codebase/AID') 
'''

"\nfrom google.colab import drive\n\ndrive.mount('/content/gdrive')\n\nos.chdir('/content/gdrive/MyDrive/Courses/COMS 4995-6 - Applied Computer Vision/Group Project/Codebase/AID') \n"

## Parameters

In [7]:
dir_hr = '../../Data/AID/AIDx1/'
dir_lr = ['../../Data/AID/AIDx8/','../../Data/AID/AIDx4/','../../Data/AID/AIDx8/','../../Data/AID/AIDx4/']
dir_out = '../../Data/AID/Outputs2/'
scale = [8,4,8,4]


In [8]:
str_case = ['EDSRp4_eval8xto1x','EDSRp4b_eval4xto1x','EDSRp8x-Bicubic_X8_L1loss_lr1e4_04182023','EDSRp4x-Bicubic_X4_L1loss_lr1e4_04172023']
str_case2 = ['8xto1xs','4xto1xs','8xto1x','4xto1x']

In [9]:
csv_train = '../../Data/AID/Train1_5class/train_HR.csv'
csv_valid = '../../Data/AID/Train1_5class/valid_HR.csv'
csv_test = '../../Data/AID/Test1_2class/test_HR.csv'

## Compute quantitative features

In [ ]:
for str_test in ['test','valid']:
    csv_file = eval('csv_' + str_test)
    list_files = np.genfromtxt(csv_file,dtype='str')
    for i,case in enumerate(str_case):
        # Set up the paths to the dataset directory and the output file
        dir_res = dir_out 
        output_file = dir_res + str_case2[i] + '_' + str_test + '_result.csv'

        # Initialize the LPIPS loss function
        loss_fn_vgg = lpips.LPIPS(net='vgg')

        # Iterate over all image pairs in the dataset directory and compute LPIPS, PSNR, and SSIM for each pair
        with open(output_file, 'w') as f:
            for file in tqdm(list_files):
              ind = [m.start() for m in list(re.finditer(r'/',file))]

              img_HR = Image.open(file)
              img_LR = Image.open(dir_lr[i] + file[ind[-2]+1:]).resize((600,600))
              file2 = dir_res + str_case[i] + '/2xto1x_' + str_test + file[ind[-1]:]
              img_SR = Image.open(file2)

              # Convert images to tensors
              img1_tensor = torch.tensor(np.array(img_HR)).permute(2, 0, 1).unsqueeze(0).float() / 255.0
              img2_tensor = torch.tensor(np.array(img_SR)).permute(2, 0, 1).unsqueeze(0).float() / 255.0
              img3_tensor = torch.tensor(np.array(img_LR)).permute(2, 0, 1).unsqueeze(0).float() / 255.0

              # Compute LPIPS distance
              lpips_distance = loss_fn_vgg(img1_tensor, img2_tensor).item()
              lpips_distance2 = loss_fn_vgg(img1_tensor, img3_tensor).item()

              # Compute PSNR
              img1_np = np.array(img_HR)
              img2_np = np.array(img_SR)
              img3_np = np.array(img_LR)
              psnr = peak_signal_noise_ratio(img1_np, img2_np)
              psnr2 = peak_signal_noise_ratio(img1_np, img3_np)

              # Compute SSIM
              ssim_value = ssim(img1_np, img2_np, multichannel=True)
              ssim_value2 = ssim(img1_np, img3_np, multichannel=True)

              # Write the results to the output file
              f.write(f"{file},{file2},{lpips_distance},{psnr},{ssim_value},{lpips_distance2},{psnr2},{ssim_value2}\n")
        
        

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\Athrun\anaconda3\lib\site-packages\lpips\weights\v0.1\vgg.pth


  0%|                                                                                          | 0/710 [00:00<?, ?it/s]C:\Users\Athrun\AppData\Local\Temp\ipykernel_20960\410159584.py:39: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(img1_np, img2_np, multichannel=True)
C:\Users\Athrun\AppData\Local\Temp\ipykernel_20960\410159584.py:40: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value2 = ssim(img1_np, img3_np, multichannel=True)
100%|██████████████████████████████████████████████████████████████████████████████| 710/710 [1:04:45<00:00,  5.47s/it]


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: C:\Users\Athrun\anaconda3\lib\site-packages\lpips\weights\v0.1\vgg.pth


 70%|███████████████████████████████████████████████████████▋                        | 494/710 [44:41<19:17,  5.36s/it]